<a href="https://colab.research.google.com/github/Maruf-16203091/Feedforward-neural-network-MLP-/blob/main/feedforwardCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
# Load your CSV file
data_path = '/content/drive/MyDrive/ML_data/Iris.csv'
df = pd.read_csv(data_path)

# Split features and labels
X = df.drop('Species', axis=1).values
y = df['Species'].values


In [19]:
le = LabelEncoder()
df['Species'] = le.fit_transform(df['Species'])

In [20]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # long for classification

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create loaders
train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [28]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)   # Input → Hidden
        self.relu = nn.ReLU()                           # Activation
        self.fc2 = nn.Linear(hidden_size, num_classes)  # Hidden → Output

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [29]:
input_size = X.shape[1]
hidden_size = 16
num_classes = len(set(y))  # number of output classes

model = SimpleNN(input_size, hidden_size, num_classes)


In [33]:
criterion = nn.CrossEntropyLoss()  # good for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [34]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for inputs, labels in train_loader:
        outputs = model(inputs)            # forward pass
        loss = criterion(outputs, labels)  # compute loss

        optimizer.zero_grad()              # clear previous gradients
        loss.backward()                    # backpropagation
        optimizer.step()                   # update weights

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/20], Loss: 1.0248
Epoch [2/20], Loss: 0.8647
Epoch [3/20], Loss: 0.7124
Epoch [4/20], Loss: 0.5715
Epoch [5/20], Loss: 0.4750
Epoch [6/20], Loss: 0.4072
Epoch [7/20], Loss: 0.3558
Epoch [8/20], Loss: 0.2994
Epoch [9/20], Loss: 0.2656
Epoch [10/20], Loss: 0.2253
Epoch [11/20], Loss: 0.1887
Epoch [12/20], Loss: 0.1560
Epoch [13/20], Loss: 0.1306
Epoch [14/20], Loss: 0.1110
Epoch [15/20], Loss: 0.0892
Epoch [16/20], Loss: 0.0784
Epoch [17/20], Loss: 0.0688
Epoch [18/20], Loss: 0.0584
Epoch [19/20], Loss: 0.0514
Epoch [20/20], Loss: 0.0450


In [35]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")


Accuracy: 100.00%
